In [ ]:
%pylab inline 

In [ ]:
# imports 
import gym
import sys
sys.path.append('/home/peter/code/projects')
sys.path.append('C:\\users\\spook\\dropbox\\code\\projects')
from gym import error, spaces, utils
from gym.utils import seeding
from collections import Counter
import time
import progressbar as pb
from tqdm import tqdm
import os
import shutil
import tensorflow as tf
#import aidevutil.denoise as denoise
from empyrical import sortino_ratio, calmar_ratio, omega_ratio
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, ActorCriticPolicy, FeedForwardPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, VecEnv, VecEnvWrapper
from stable_baselines import A2C, PPO2, DQN, ACKTR, ACER
from stable_baselines.common.vec_env import DummyVecEnv
from trading_env import TradingEnv

In [ ]:
np.random.seed()

In [ ]:
dataname = 'eurusd_1m_2015'

input_source = np.load("data/data_%s.npy" % dataname)
to_predict = np.load("data/data_%s_targets.npy" % dataname)

to_predict = to_predict[3,:].reshape(-1)

input_source = input_source.T

In [ ]:
input_source.shape, to_predict.shape

In [ ]:
# lookahead bias on purpose
#input_source[:, 22] = np.roll(input_source[:, 22], -3)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
#k = SelectKBest(score_func=f_regression, k=16)
#k.fit(input_source, to_predict)
#input_source = k.transform(input_source)

In [ ]:
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]

In [ ]:
input_source.shape, to_predict.shape

In [ ]:
bars_per_episode = 1000
winlen = 1
traded_amt = 100
initial_balance = 1000000000000
commission = 0.0
slippage = 0.0

In [ ]:
# multiprocess environment
n_cpu = 4
env = SubprocVecEnv([lambda: TradingEnv(input_source, to_predict,
                 winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt, initial_balance=initial_balance,
                 commission=commission, slippage=slippage,
                 reward_type='cur_balance',
                 min_ratio_trades = 20,
                 max_position_time = 30,
                 ) for i in range(n_cpu)])
#env = TradingEnv(input_source, to_predict,
#                 winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt, initial_balance=initial_balance,
#                 commission=commission, slippage=slippage,
#                 reward_type='sortino',
#                 min_ratio_trades = 50,
#                 )
#env = DummyVecEnv([lambda: env])18

t = 0
# Custom MLP policy of two layers of size 32 each with tanh activation function
#policy_kwargs = dict(act_fun=tf.nn.tanh, net_arch=[128, 128])

#[shutil.rmtree('/home/peter/tblog/'+x) for x in os.listdir('/home/peter/tblog/') if x]
# model = PPO2(MlpPolicy, env, n_steps=32, verbose=0, #nminibatches=1, 
#              policy_kwargs=policy_kwargs, 
#              gamma=0.99,#0.99,
#              ent_coef=0.01,#0.01,
#              learning_rate=0.0005,
#              vf_coef=0.5,
#              max_grad_norm=0.5,
#              lam=0.95,
#              tensorboard_log='/home/peter/tblog')
model = ACKTR(MlpPolicy, env, verbose=0, 
#              policy_kwargs=policy_kwargs, 
              tensorboard_log='/home/peter/tblog')

In [ ]:
try:
    model.learn(total_timesteps=100_000_000)
except KeyboardInterrupt:
    pass

In [ ]:
#model.save('model.pkl')

In [ ]:
input_source.shape, test_input_source.shape

In [ ]:
#model.load('model.pkl')

In [ ]:
env = TradingEnv(test_input_source, test_to_predict,
                 winlen=winlen, bars_per_episode=12000, traded_amt=traded_amt,
                 commission=commission, slippage=slippage,
                 initial_balance=initial_balance,
                 reward_type='cur_balance',
                 min_ratio_trades = 20,
                 max_position_time = 30,
                 )
env = DummyVecEnv([lambda: env])


nstate = model.initial_state  # get the initial state vector for the reccurent network
#dones = np.zeros(nstate.shape[0])  # set all environment to not done
#nstate=None

observation = env.envs[0].reset()#env.reset()
done = False
navs = []
acts = []
for i in tqdm(range(env.envs[0].bars_per_episode)):
    action, nstate = model.predict([observation], state=nstate, deterministic=1)
    acts.append(action)
    observation, reward, done, info = env.envs[0].step(action)#env.step(action)
    if done:
        break
navs = env.envs[0].returns
kl = []
t = 0
for n in navs:
    t = t + n
    kl.append(t)
plot(kl);

In [ ]:
len(navs)

In [ ]:
# calculate the likelihood of success for any given episode
try:
    l = 10000

    krl = []
    p = pb.ProgressBar(max_value=l)
    for i in range(l):
        p.update(i)
        observation = env.envs[0].reset()
        nstate = model.initial_state
        done = False
        navs = []
        for i in (range(env.envs[0].bars_per_episode)):
            action, nstate = model.predict([observation], state=nstate, deterministic=1)
            acts.append(action)
            observation, reward, done, info = env.envs[0].step(action)#env.step(action)
            if done:
                break
        krl.append(sum(env.envs[0].returns ))
    p.finish()
except KeyboardInterrupt:
    pass

krl = np.array(krl)
print('Profit likelihood: %3.3f%%' % (100*(sum(krl > 0) / len(krl))))

In [ ]:
hist(krl,50);

In [ ]:
sum(env.envs[0].returns)